In [1]:
!pip install matplotlib
import matplotlib.pyplot as plt

## Exercise 1: Hello world

#### 1.Write an MPI program which prints the message "Hello World"

In [2]:
%%file exercice11.py
from mpi4py import MPI
COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size ()
RANK = COMM.Get_rank()
print(" Hello World") 

Overwriting exercice11.py


In [3]:
!mpirun -n 4 python3 exercice11.py

 Hello World
 Hello World
 Hello World
 Hello World


##### 2.Modify your program so that each process prints out both its rank and the total number of processes P that the code is running on, i.e. the size of MPI_COMM_WORLD.

In [4]:
%%file exercice12.py
from mpi4py import MPI
COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()
print(f"  I\' am  process {RANK} among {SIZE} processes")

Overwriting exercice12.py


In [5]:
!mpirun -n 4 python3 exercice12.py

  I' am  process 2 among 4 processes
  I' am  process 0 among 4 processes
  I' am  process 3 among 4 processes
  I' am  process 1 among 4 processes


#### 3.Modify your program so that only a single controller process (e.g. rank 0) prints out a message (very useful when you run with hundreds of processes).

In [6]:
%%file exercice13.py
from mpi4py import MPI
COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()
if RANK == 0:
    print(f"  I\' am  process {RANK} among {SIZE} processes")

Overwriting exercice13.py


In [7]:
!mpirun -n 4 python3 exercice13.py

  I' am  process 0 among 4 processes


## Exercise 2:

In [8]:
%%file exercice2.py
from mpi4py import MPI

COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()

while True:
    if RANK == 0:%%file exercice2.py
from mpi4py import MPI

COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()
        number = int(input('Enter a number (negative integer to stop): '))
        if number < 0:
            for i in range(1, SIZE):
                COMM.send(number, dest=i, tag=1)
            break
        for i in range(1, SIZE):
            COMM.send(number, dest=i, tag=1)
    else: 
        number = COMM.recv(source=0, tag=1)
        if number < 0:
            break
        print(f'Process {RANK} got {number}')
print(f'Process {RANK} stopped')

Overwriting exercice2.py


In [9]:
%%file exercice2.py
from mpi4py import MPI

COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()
Valeurs =[]
if RANK == 0:
    while   True:
        numbre = int(input("entrer un nombre entier postive :"))
        if numbre <0:
            break
else :
    numbre = None    
recvbuf = COMM . bcast ( numbre , root=0 )
#Valeurs.append(recvbuf)
print ("Je suis le process",{RANK} ," J'ai réssu la valeur ",{recvbuf})

    

Overwriting exercice2.py


In [11]:
!mpirun -n 4 python3 exercice2.py

entrer un nombre entier postive :^C


## Exercise 3: Ping-Pong

In [12]:
%%file exercice3.py
from mpi4py import MPI
import random

COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()
it_max = 5

if SIZE != 2:
    print('This program only works with 2 processes')
    exit()

if RANK == 0:
    #x =  int(input("entrer un nombre entier postive :")) 
    x= 0
    COMM . send ( x , dest=1 )
    #print(f"Iteration 1 : Process {RANK} sending {x} to Process 1")
    
for i in range(it_max):
    if RANK == 1:
        x = COMM.recv ( source=0 )
        print(f"Iteration {i+1}: Process {RANK} received {x} from Process 0")
        x +=1
        COMM . send ( x , dest= 0)
        #print(f"Iteration {i+1}: Process {RANK} sending {x} to Process 0")

    if RANK==0:
        x = COMM.recv(source=1)
        print(f"Iteration {i+1}: Process {RANK} received {x} from Process 1")
        x +=1
        COMM . send ( x , dest=1 )
        #print(f"Iteration {i+1}: Process {RANK} sending {x} to Process 1")
   


Overwriting exercice3.py


In [13]:

!mpirun -n 2 python3 exercice3.py

Iteration 1: Process 1 received 0 from Process 0
Iteration 1: Process 0 received 1 from Process 1
Iteration 2: Process 0 received 3 from Process 1
Iteration 3: Process 0 received 5 from Process 1
Iteration 4: Process 0 received 7 from Process 1
Iteration 5: Process 0 received 9 from Process 1
Iteration 2: Process 1 received 2 from Process 0
Iteration 3: Process 1 received 4 from Process 0
Iteration 4: Process 1 received 6 from Process 0
Iteration 5: Process 1 received 8 from Process 0


## Exercise 4: Sending in a ring (Broadcast by ring)

In [14]:
%%file exercice4.py
from mpi4py import MPI
import random

COMM= MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()
if RANK==0:
    data = 10
    COMM.send(data, dest=1)  
    
if 0 <  RANK <SIZE:
    data= COMM.recv(source = RANK -1)
    print(f" Process {RANK} received {data} from Process {RANK-1}")
    data +=RANK
    if RANK!= SIZE -1:
        COMM.send(data,  dest= RANK+1)   

    

Overwriting exercice4.py


In [15]:
!mpirun -n 4 python3 exercice4.py

 Process 1 received 10 from Process 0
 Process 2 received 11 from Process 1
 Process 3 received 13 from Process 2


## Exercise 5: Solving 1D advection equation

In [16]:
%%file exercice5.1.py
from mpi4py import MPI
import numpy as np
import matplotlib.pyplot as plt
import time

def solve_1d_linearconv(u, un, nt, nx, dt, dx, c, comm):
    for n in range(nt):
        un[:] = u[:]
        for i in range(1, nx):
            u[i] = un[i] - c * dt / dx * (un[i] - un[i-1])
        if comm.Get_rank() != 0:
            comm.send(u[1], dest=comm.Get_rank() - 1)
            u[0] = comm.recv(source=comm.Get_rank() - 1)
        if comm.Get_rank() != comm.Get_size() - 1:
            comm.send(u[-2], dest=comm.Get_rank() + 1)
            u[-1] = comm.recv(source=comm.Get_rank() + 1)
    return u

if __name__ == "__main__":
    start_time = time.time()
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()
    nx = 200
    nt = 100 
    c = 2
    dx = 2 * np.pi / (nx - 1)
    CFL = 0.8
    dt = CFL * dx / c
    x = np.linspace(0, 2 * np.pi, nx)

    local_nx = nx // size
    remainder = nx % size
    if rank < remainder:
        local_nx += 1

    start_idx = rank * local_nx + min(rank, remainder)
    end_idx = start_idx + local_nx
    if rank < remainder:
        end_idx += 1

    u = np.zeros(local_nx)
    u[:] = np.where((x[start_idx:end_idx] >= 3) & (x[start_idx:end_idx] <= 4), 1, 0)
    un = np.zeros(local_nx)
    print(f'Process {rank}: u = {u}')

    u = solve_1d_linearconv(u, un, nt, local_nx, dt, dx, c, comm)

    # Gather results on root process
    if rank == 0:
        all_results = np.empty(nx)
        all_results[start_idx:end_idx] = u
        for i in range(1, size):
            start_idx_i = i * local_nx + min(i, remainder)
            end_idx_i = start_idx_i + local_nx
            if i < remainder:
                end_idx_i += 1
            u_i = np.empty(local_nx)
            comm.Recv(u_i, source=i)
            all_results[start_idx_i:end_idx_i] = u_i
        end_time = time.time()
        print(f"Process {rank}: {all_results}")
        print("Time taken:", end_time - start_time, "seconds")
        plt.plot(x, all_results)
        plt.show()
        plt.savefig("plot.png")
    else:
        comm.Send(u, dest=0)




Overwriting exercice5.1.py


In [17]:
!mpirun -n 4 python3 exercice5.1.py

Process 2: u = [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
Process 0: u = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
Process 1: u = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1.]
Process 3: u = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
Process 0: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000